<p style="text-align:right";>Polytech Nantes - 2018</p>

# Objectif & Plan

La bibliothèque [scikit-learn](https://scikit-learn.org/) est destinée à l'apprentissage automatique et l'analyse de données. L'objectif de cette séance est d'introduire quelques méthodes et méthodologies pour ce domaine.

Le plan de ce notebook est le suivant :
1. le jeu de données Iris
2. modèle de classification
3. modèle de régression
4. évaluation de modèle

# Le jeu de données Iris

Le jeu de données Iris décrit les variations morphologiques des fleurs de trois espèces d'iris (*Iris setosa, Iris virginica et Iris versicolor*). Il comprend 150 fleurs décrites par  la longueur et la largeur des sépales et des pétales, en centimètres. Le but est de reconnaître l'espèce à partir des caractéritiques de la fleur. Comme il s'agit d'un jeu de données standard, il est proposé dans [scikit-learn](https://scikit-learn.org/) et peut être chargé ainsi :

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
type(iris)

Afin de stocker le jeu de données dans un dataFrame [pandas](http://pandas.pydata.org/), exécutez :

In [ ]:
import pandas as pd

df = pd.DataFrame(iris['data'], columns=iris['feature_names'])
df['target'] = iris['target']
df.head()

Dans la dernière colonne, les espèces *Setosa*, *Versicolour* et *Virginica* sont codées respectivement par 0, 1 et 2.

On peut renommer les variables en chaînes sans espace pour pouvoir les manipuler directement à l'aide de la syntaxe `df.nomDeColonne` :

In [ ]:
df.columns=("longueurSépale","largeurSépale", "longueurPétale", "largeurPétale", "target")
df.head()

#### Exercice

Vérifiez les caractéristiques du dataFrame à l'aide de `shape`, `dtypes` et `count()` (voir notebook sur [VisuStats](https://gitlab.univ-nantes.fr/pigeau-a/peipMadoc2018/tree/master/VisuStats)).

#### Exercice

En vous aidant du notebook [VisuStats](https://gitlab.univ-nantes.fr/pigeau-a/peipMadoc2018/tree/master/VisuStats), faites l'analyse des 4 variables numériques (tableaux d'effectifs, histogrammes, boîtes à moustaches, indicateurs) et l'analyse de la variable catégorique `target` (tableau d'effectifs, diagramme en secteurs ou en barres).

# Modèle de classification

Vous allez construire plusieurs modèles pour prédire/expliquer/reconnaître l'espèce à partir des autres variables. Pour construire un modèle prédictif, il faut appliquer un algorithme d'apprentissage supervisé. Vous trouverez dans la doc de [scikit-learn](https://scikit-learn.org/) une description des différents algorithmes pris en charge dans la bibliothèque et des fonctions correspondantes. Il existe des algorithmes pour prédire des variables cibles catégoriques (algorithmes de classification) et d'autres pour prédire des variables cibles numériques (algorithmes de régression). Dans le cas des données Iris, s'agit-il de classification ou de régression ?

Dans la suite, nous illustrons sur un arbre de décision utiliser  ...  pour les diféfrents types de modèles ce sera les mêmes fonctions

### Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

Vous avez créé un modèle, mais pour le moment seul l'algorithme d'apprentissage a été spécifié, le modèle n'a pas encore été confronté aux données. Pour réaliser l'apprentissage du modèle à partir des données (on dit aussi ajuster le modèle aux données), on utilise la fonction `fit()`. Cette fonction prend en paramètres deux tableaux de données :
- un tableau `y` qui contient uniquement la variable à prédire (cible)
- un tableau `X` qui contient les variables utiles pour faire les prédictions sur y

#### Exercice
Sur les données Iris, construisez ces deux tableaux :

In [ ]:
y=...

In [ ]:
X=...

Vous pouvez maintenant réaliser l'apprentissage :

In [ ]:
model=model.fit(X, y)

Remarquez que dans cet exemple simplifié, nous avons laissé tous les paramètres de l'algorithme d'apprentissage à leurs valeurs par défaut :

In [ ]:
model

Vous pouvez obtenir les prédictions du modèle à l'aide de la fonction `predict()` :

Vous allez maintenant utiliser le modèle pour prédire un nouvel individu (ici donc une fleur d'iris que vous allez imaginer). Commencez par décrire de nouveaux individus dans un dataFrame :

In [ ]:
descriptionIndividus = {'longueurSépale': [3, 6], 'largeurSépale': [4, 3], 'longueurPétale': [2, 1.5], 'largeurPétale': [0.8, 2]}
nouveauDataFrame = pd.DataFrame(data=descriptionIndividus)
nouveauDataFrame

In [ ]:
model.predict(nouveauDataFrame)

#### Exercice

Crééz un individu correspondant à la fleur d'iris moyenne de la classe 0, puis vérifiez que le modèle prédit bien la bonne classe pour cette fleur. Vérifiez également pour les autres classes.

En projetant les données dans un espace (typiquement un plan), il est possible de visualiser les prédictions du modèles. Ci-dessous nous réduisons le jeu de données uniquement aux sépales, puis construisons le modèle et visualisons les "surfaces de décision" de l'arbre de décisions. 

In [ ]:
# Données
colonnes=[0, 1]   #choix des deux premières colonnes (longueur et largeur des sépales)
Xreduit=df.iloc[:,colonnes]
y=df.iloc[:,4]

# Construction du modèle sur les données réduites aux deux colonnes choisies
modelReduit=DecisionTreeClassifier()
modelReduit=modelReduit.fit(Xreduit, y)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Changement de la taille par défaut pour tous les graphes :
plt.rcParams["figure.figsize"] = [10,10]
 
# Parameters
n_classes = 3
plot_colors = "ryb"
plot_step = 0.02


x_min, x_max = Xreduit.iloc[:, 0].min() - 1, Xreduit.iloc[:, 0].max() + 1
y_min, y_max = Xreduit.iloc[:, 1].min() - 1, Xreduit.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))
plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

Z = modelReduit.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)

plt.xlabel(iris.feature_names[colonnes[0]])
plt.ylabel(iris.feature_names[colonnes[1]])

# Plot the training points
for i, color in zip(range(n_classes), plot_colors):
    idx = np.where(y == i)
    plt.scatter(Xreduit.iloc[list(idx[0]),0], Xreduit.iloc[list(idx[0]),1], c=color, label=iris.target_names[i],
                cmap=plt.cm.RdYlBu, edgecolor='black', s=25)

plt.suptitle("Prédictions par arbre de décision\n sur les iris décrits uniquement par les sépales (longueur et largeur)", fontsize=16)
plt.legend(loc='lower right', borderpad=0, handletextpad=0, fontsize=16)
plt.axis("tight")
plt.show()

#### Exercice

Construisez un individu (une fleur d'iris) pour laquelle la prédiction du modèle (`modèleRéduit` ci-dessus) vous semble sujette à caution. Faites la prédiction.

Un modèle ne sert à rien si l'on n'a pas une idée de ses performances. Revenons au modèle complet (construit sur toutes les données) pour l'évaluer.

In [ ]:
X=df.iloc[:,:4]
y=df.iloc[:,4]
model = DecisionTreeClassifier()
model=model.fit(X, y)

Vous pouvez obtenir le taux de réussite de modèle à l'aide de la fonction `score()` :

In [ ]:
model.score(X,y)

#### Exercice
1. Le modèle est-il bon ?
2. Vérifiez ce taux de réussite en affichant les prédictions du modèle du les données complètes `X` avec `predict()`.
3. Proposez une manière plus pertinente d'évaluer les performances du modèle.

## Autres modèles de classification

#### Exercice

Essayez avec d'autres algorithmes comme les [k plus proches voisins](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) et les [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). A chaque fois :
1. construisez le modèle,
2. donnez son taux de réussite sur un ensemble de test,
3. faite le modèleRéduit (uniquement sur les sépales) et visualisez.

# Modèle de Régression

A la différence de la classification, qui permet de déterminer l'appartenance d'une donnée à un groupe, la régression permet de prédire la valeur d'une donnée continue :
- la classification consiste à prédire une donnée discrète
- la régression consiste à prédire une quantité continue 

Par exemple, la prédiction de la valeur d'un bien immobilier utilisera plutôt une méthode de régression qu'une méthode  classification.

Dans cette partie, la colonne `target` n'est donc plus nécessaire.

#### Exercice

1. supprimez la colonne `target` des données
2. A l'aide des méthodes vues dans le notebook [VisuStats](https://gitlab.univ-nantes.fr/pigeau-a/peipMadoc2018/tree/master/VisuStats), recherchez les deux variables les plus corrélées dans le jeu de données.
3. Essayez de prédire l'une à partir de l'autre avec la [régression linéaire](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) et les [k plus proches voisins](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
4. Utilisez `score()` pour évaluer les modèles

# Evaluation de modèle

Pour le moment, nos modèles ont été évalués en utilisant le jeu de données qui a été utilisé pour construire le modèle. Mais dans la vraie vie, nous ne connaissons malheureuement pas en pratique les nouvelles données qui vont devoir être évaluées. Afin de mieux évaluer la qualité de nos modèle, il faut donc une méthode plus robuste de construction / évaluation. Nous proposons ici la méthode de __validation croisée__.

L'apprentissage d'un modèle nécessite un ensemble de données d'apprentissages et un ensemble de données de test pour évaluer le modèle obtenu. Afin de vérifier que la méthode de génération du modèle est robuste dans le cas où il est utilisé sur de nouvelles données, il est nécessaire de l'évaluer plusieurs fois sur des données d'apprentissage / test différentes. La validation croisée est une méthode pour réaliser cette vérification.

L'approche consiste à choisir aléatoirement X%, appelées __données d'apprentissage__, des données pour apprendre le modèle et (100% - X%) des données restantes, appelées __données de test__, pour évaluer le modèle. La prédiction du modèle est réalisée sur les données de test et un score de qualité est obtenu. En réalisant plusieurs fois le tirag aléatoire de données d'apprentissage et de test, on peut obtenir ainsi une moyenne du score sur un jeu de données. 

La validation croisée permet ainsi de dire que _sur ce jeu de données et avec notre modèle, nous obtenons en moyenne tel score de prédiction_. Nous limitons le risque d'avoir un score de prédiction très élevé mais obtenu simplement sur un jeu de données très précis. 

Pour conclure, nous tentons de garantir que notre modèle soit plus robuste, dans le cas où il recevra de nouvelles données.

#### Exercice :
1. documentez vous sur la [validation croisée](https://fr.wikipedia.org/wiki/Validation_crois%C3%A9e) et sur son implémentation dans la bibliothèque sklearn ([sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)) 
2. reprenez les modèles testés précédemment et implémentez maintenant la méthode de validation croisée, en lançant 10 tests de 80% / 20% 
3. affichez la moyenne et la variance des scores obtenus
4. est ce que la comparaison des modèles utilisez le précédent bloc de code pour comparer les différents modèles de prédiction


In [ ]:
from sklearn.model_selection import train_test_split

# on divise les données entre ensemble de train (apprentissage) et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# on construit le modèle sur l'ensemble de train
model = ...   # ou : modele = DecisionTreeClassifier(min_samples_leaf=20)
model = model.fit(X_train, y_train)

# on évalue le modèle sur l'ensemble de test
model.score(X_test, y_test)


## Partie 2 - Projet Immobilier

Pour les questions suivantes, un nouveau notebook doit être créé. 

#### Exercice :
1. sélectionnez les variables vous semblant utiles pour la prédiction du prix d'une maison ou d'un appartement
2. appliquez les méthodes de [régression linéaire](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) ou les [k plus proches voisins](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) sur les données immobilières (n'oubliez de faire l'évaluation via la validation croisée)
3. documentez vous sur les modèles basés sur les [réseaux de neurones](https://scikit-learn.org/stable/modules/neural_networks_supervised.html) 
4. appliquez la méthode basée sur les réseaux de neurones
5. testez les différents ensembles de variables d'apprentissages à utiliser pour la prédiction
6. construisez, si cela n'est pas déjà fait, un modèle de prédiction commun pour les appartements et les maisons
7. construisez, si cela n'est pas déjà fait, un modèle pour les appartements et un autre pour les maisons
8. comparez les résultats de chaque modèle